### Carga de librerias

In [25]:
import pandas as pd
import glob
import re
import unicodedata
from rapidfuzz import fuzz


### Carga de archivos y visualización

In [26]:
"""
Carga y combina múltiples archivos CSV desde una carpeta específica,
saltando filas de encabezado no necesarias y líneas de pie de página.

Este script:
1. Busca todos los archivos CSV en la ruta especificada.
2. Lee cada archivo ignorando las primeras 25 filas y las últimas 5 líneas.
3. Combina todos los DataFrames en uno solo.
4. Muestra el tamaño final del DataFrame combinado.
"""

# Ruta a los archivos CSV dentro de la carpeta "mineduc"
ruta = "mineduc/*.csv"

# Leer y procesar cada archivo CSV encontrado en la ruta:
# - skiprows=25 → ignora las primeras 25 filas (encabezados no útiles).
# - skipfooter=5 → ignora las últimas 5 filas (información no necesaria).
# - engine="python" → permite usar skipfooter (no disponible con el motor por defecto).
dfs = [
    pd.read_csv(archivo, skiprows=25, skipfooter=5, engine="python")
    for archivo in sorted(glob.glob(ruta))
]

# Unir todos los DataFrames en uno solo
# ignore_index=True → reinicia el índice para que sea continuo.
df_final = pd.concat(dfs, ignore_index=True)

# Mostrar la forma final del DataFrame (filas, columnas)
print(df_final.shape)


(6600, 17)


In [27]:
"""
Visualización rápida del DataFrame resultante `df_final`.

Dependiendo de la función utilizada, se obtiene:
- `df_final` → Muestra todo el DataFrame (no recomendable si es muy grande).
- `df_final.head()` → Muestra las primeras 5 filas.
- `df_final.tail()` → Muestra las últimas 5 filas.
- `df_final.info()` → Muestra un resumen: número de filas, columnas, tipo de datos y valores no nulos.
- `df_final.describe(include="all")` → Muestra estadísticas descriptivas de todas las columnas.
"""

# Ejemplo: ver las primeras 5 filas del DataFrame
df_final.head()

# Ejemplo: ver información general del DataFrame
df_final.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6600 entries, 0 to 6599
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CODIGO           6600 non-null   object
 1   DISTRITO         6600 non-null   object
 2   DEPARTAMENTO     6600 non-null   object
 3   MUNICIPIO        6600 non-null   object
 4   ESTABLECIMIENTO  6600 non-null   object
 5   DIRECCION        6598 non-null   object
 6   TELEFONO         6554 non-null   object
 7   SUPERVISOR       6600 non-null   object
 8   DIRECTOR         6574 non-null   object
 9   NIVEL            6600 non-null   object
 10  SECTOR           6600 non-null   object
 11  AREA             6600 non-null   object
 12  STATUS           6600 non-null   object
 13  MODALIDAD        6600 non-null   object
 14  JORNADA          6600 non-null   object
 15  PLAN             6600 non-null   object
 16  DEPARTAMENTAL    6600 non-null   object
dtypes: object(17)
memory usage: 876.7

### Formato CODIGO

In [28]:
"""
Verificación del formato de la columna 'CODIGO' en el DataFrame `df_final`.

Este código:
1. Define un patrón (regex) que valida que el código siga el formato NN-NN-NNNN-NN
   - NN: dos dígitos (ejemplo: 16)
   - NN: dos dígitos (ejemplo: 01)
   - NNNN: cuatro dígitos (ejemplo: 0138)
   - NN: dos dígitos (ejemplo: 46)
2. Filtra las filas que NO cumplen con este formato.
3. Imprime dichas filas para su revisión.
"""

# 1. Compilar patrón regex para el formato NN-NN-NNNN-NN
pattern = re.compile(r"^\d{2}-\d{2}-\d{4}-\d{2}$")

# 2. Filtrar las filas donde 'CODIGO' no cumple el patrón:
# - Se convierte a string para evitar errores con valores numéricos.
# - Se aplica la función match del patrón, que devuelve True si cumple.
# - El operador ~ invierte el booleano para quedarse con los casos incorrectos.
filas_incorrectas = df_final[~df_final["CODIGO"].astype(str).apply(lambda x: bool(pattern.match(x)))]

# 3. Mostrar las filas con formato incorrecto
print(filas_incorrectas)


Empty DataFrame
Columns: [CODIGO, DISTRITO, DEPARTAMENTO, MUNICIPIO, ESTABLECIMIENTO, DIRECCION, TELEFONO, SUPERVISOR, DIRECTOR, NIVEL, SECTOR, AREA, STATUS, MODALIDAD, JORNADA, PLAN, DEPARTAMENTAL]
Index: []


### Formato DISTRITO 

In [29]:
"""
Verificación del formato de la columna 'DISTRITO' en el DataFrame `df_final`.

Este código:
1. Define un patrón (regex) para validar que el valor del distrito siga el formato NN-NNN:
   - NN: dos dígitos (ejemplo: 19)
   - NNN: tres dígitos (ejemplo: 015)
   - Ejemplo completo válido: 19-015
2. Filtra todas las filas cuyo valor en 'DISTRITO' no cumpla con este formato.
3. Imprime dichas filas para su revisión.

Uso:
- Permite detectar registros con formato incorrecto en el código de distrito.
"""

# 1. Compilar patrón regex para el formato NN-NNN
pattern = re.compile(r"^\d{2}-\d{3}$")

# 2. Filtrar filas con formato incorrecto:
# - Convertir a string para evitar errores si hay valores numéricos.
# - Usar pattern.match() para verificar coincidencia exacta.
# - El operador ~ invierte el booleano para quedarse con los que NO cumplen.
filas_incorrectas = df_final[~df_final["DISTRITO"].astype(str).apply(lambda x: bool(pattern.match(x)))]

# 3. Mostrar las filas incorrectas
print(filas_incorrectas)


Empty DataFrame
Columns: [CODIGO, DISTRITO, DEPARTAMENTO, MUNICIPIO, ESTABLECIMIENTO, DIRECCION, TELEFONO, SUPERVISOR, DIRECTOR, NIVEL, SECTOR, AREA, STATUS, MODALIDAD, JORNADA, PLAN, DEPARTAMENTAL]
Index: []


### Formato ESTABLECIMIENTO

In [30]:
"""
Verificación de mayúsculas en la columna 'ESTABLECIMIENTO' del DataFrame `df_final`.

Este código:
1. Compara el valor original de cada fila en la columna 'ESTABLECIMIENTO'
   con su versión en mayúsculas (`.str.upper()`).
2. Si el valor original es diferente de su versión en mayúsculas, significa
   que contiene letras en minúscula y, por lo tanto, no cumple con el requisito
   de estar totalmente en mayúsculas.
3. Filtra y guarda esas filas en `filas_incorrectas`.
4. Imprime las filas encontradas para revisión.

Uso:
- Permite detectar nombres de establecimientos que no están escritos
  completamente en mayúsculas, para normalizarlos después.
"""

# 1. Filtrar filas donde el valor original no coincide con su versión en mayúsculas
filas_incorrectas = df_final[df_final["ESTABLECIMIENTO"] != df_final["ESTABLECIMIENTO"].str.upper()]

# 2. Mostrar las filas que no cumplen con el formato en mayúsculas
print(filas_incorrectas)



Empty DataFrame
Columns: [CODIGO, DISTRITO, DEPARTAMENTO, MUNICIPIO, ESTABLECIMIENTO, DIRECCION, TELEFONO, SUPERVISOR, DIRECTOR, NIVEL, SECTOR, AREA, STATUS, MODALIDAD, JORNADA, PLAN, DEPARTAMENTAL]
Index: []


In [31]:

"""
Reemplazo de comillas simples por comillas dobles en la columna 'ESTABLECIMIENTO'
del DataFrame `df_final`, sin afectar las comillas que forman parte de una palabra.

Este código:
1. Convierte los valores de la columna 'ESTABLECIMIENTO' a tipo string
   para evitar errores al aplicar expresiones regulares.
2. Utiliza `str.replace()` con una expresión regular que busca comillas simples
   que no estén dentro de una palabra:
   - `(?<!\w)'` → Comilla simple precedida por algo que NO es un carácter alfanumérico.
   - `'(?!\w)` → Comilla simple seguida por algo que NO es un carácter alfanumérico.
3. Sustituye esas comillas simples encontradas por comillas dobles (`"`).

Uso:
- Permite estandarizar el uso de comillas dobles en los nombres de establecimientos,
  manteniendo correctas las comillas simples que son parte de palabras como O'BRIEN.
"""

df_final["ESTABLECIMIENTO"] = (
    df_final["ESTABLECIMIENTO"]
    .astype(str)  # Asegurar que todos los valores sean strings
    .str.replace(r"(?<!\w)'|'(?!\w)", '"', regex=True)  # Reemplazo condicional
)


In [32]:
"""
Detección de valores repetidos en la columna 'ESTABLECIMIENTO' del DataFrame `df_final`.

Este código:
1. Utiliza `.duplicated(keep=False)` para marcar como `True` todas las filas
   que tienen un valor duplicado en la columna 'ESTABLECIMIENTO', sin importar
   si son la primera o posteriores apariciones (`keep=False` marca todas).
2. Filtra el DataFrame original `df_final` para quedarse únicamente con esos registros duplicados.
3. Ordena el resultado por la columna 'ESTABLECIMIENTO' para que las repeticiones
   queden agrupadas y sean más fáciles de revisar.
4. Imprime el resultado para su análisis.

Uso:
- Sirve para identificar todos los establecimientos que aparecen más de una vez en el dataset.
"""

# 1. Filtrar las filas donde 'ESTABLECIMIENTO' aparece más de una vez
repetidos = df_final[df_final["ESTABLECIMIENTO"].duplicated(keep=False)]

# 2. Mostrar los duplicados ordenados alfabéticamente por nombre
print(repetidos.sort_values("ESTABLECIMIENTO"))



             CODIGO DISTRITO   DEPARTAMENTO      MUNICIPIO  \
4321  17-05-0335-46   17-007          PETEN    LA LIBERTAD   
4292  17-05-0211-46   17-007          PETEN    LA LIBERTAD   
4315  17-05-0319-46   17-007          PETEN    LA LIBERTAD   
4304  17-05-0260-46   17-007          PETEN    LA LIBERTAD   
5947  06-01-0118-46   06-039     SANTA ROSA        CUILAPA   
...             ...      ...            ...            ...   
392   04-01-0058-46   04-001  CHIMALTENANGO  CHIMALTENANGO   
525   04-01-3245-46   04-001  CHIMALTENANGO  CHIMALTENANGO   
520   04-01-3068-46   04-001  CHIMALTENANGO  CHIMALTENANGO   
431   04-01-0230-46   04-001  CHIMALTENANGO  CHIMALTENANGO   
473   04-01-0357-46   04-001  CHIMALTENANGO  CHIMALTENANGO   

                                        ESTABLECIMIENTO  \
4321          "CENTRO DE EDUCACION INTEGRAL EL NARANJO"   
4292          "CENTRO DE EDUCACION INTEGRAL EL NARANJO"   
4315          "CENTRO DE EDUCACION INTEGRAL EL NARANJO"   
4304          "CENT

In [34]:
"""
Detección de nombres de establecimientos con alta similitud (mayor al 95%)
utilizando la librería RapidFuzz.

Flujo del código:
1. Obtiene los nombres únicos de la columna 'ESTABLECIMIENTO' en `df_final`.
2. Compara cada nombre con los demás (sin repetir combinaciones).
3. Calcula el porcentaje de similitud usando `fuzz.ratio()`:
   - 100% significa idénticos.
   - 0% significa totalmente distintos.
4. Filtra únicamente aquellos pares cuya similitud esté entre 95% y 99%
   (muy parecidos, pero no exactamente iguales).
5. Guarda los resultados en una lista con:
   - Nombre original
   - Nombre comparado
   - Porcentaje de similitud
6. Convierte la lista en un DataFrame (`df_parecidos`).
7. Ordena el DataFrame de mayor a menor similitud.
8. Imprime cuántos pares cumplen el criterio.
9. Muestra el DataFrame resultante.

Uso:
- Este script es útil para detectar errores ortográficos menores,
  variaciones en la escritura o diferencias por espacios/puntuación
  en nombres que deberían ser iguales.
"""

from rapidfuzz import fuzz
import pandas as pd

# 1. Obtener nombres únicos
nombres = df_final["ESTABLECIMIENTO"].unique()

# 2. Lista para almacenar los resultados
resultados = []

# 3. Comparar cada nombre con los demás
for i, nombre in enumerate(nombres):
    for j in range(i + 1, len(nombres)):  # Evitar comparaciones duplicadas
        similitud = fuzz.ratio(nombre, nombres[j])  # 4. Calcular similitud
        if similitud > 95 and similitud < 100:      # Filtrar pares >95% y <100%
            resultados.append({
                "Nombre_Original": nombre,
                "Nombre_Comparado": nombres[j],
                "Similitud (%)": similitud
            })

# 5. Crear DataFrame y ordenarlo por similitud descendente
df_parecidos = pd.DataFrame(resultados).sort_values("Similitud (%)", ascending=False)

# 6. Mostrar cantidad total de coincidencias encontradas
print(f"Total de pares con similitud > 95%: {len(df_parecidos)}")

# 7. Mostrar el DataFrame con los resultados
df_parecidos


Total de pares con similitud > 95%: 879


,Nombre_Original,Nombre_Comparado,Similitud (%)
295,"COLEGIO ""CENTRO DE FORMACION PROFESIONAL PRE-U...","COLEGIO ""CENTRO DE FORMACION PROFESIONAL PREUN...",99.393939
527,COLEGIO EDUCATIVO PRIVADO MIXTO DE FORMACION I...,COLEGIO EDUCATIVO PRIVADO MIXTO DE FORMACION I...,99.363057
214,INSTITUTO NACIONAL DE EDUCACIÓN DIVERSIFICADA ...,INSTITUTO NACIONAL DE EDUCACIÓN DIVERSIFICADA ...,99.328859
328,INSTITUTO MIXTO MUNICIPAL DE EDUCACIÓN BÁSICA ...,INSTITUTO MIXTO MUNICIPAL DE EDUCACIÓN BÁSICA ...,99.310345
87,CENTRO DE ESTUDIOS TECNICOS Y AVANZADOS DE CHI...,CENTRO DE ESTUDIOS TECNICOS Y AVANZADOS DE CHI...,99.290780
...,...,...,...
95,"PROGRAMA NACIONAL DE EDUCACIÓN ALTERNATIVA, PR...",PROGRAMA NACIONAL DE EDUCACION ALTERNATIVA -PR...,95.049505
658,INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA ...,INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADO-...,95.049505
345,"PROGRAMA NACIONAL DE EDUCACIÓN ALTERNATIVA , P...",PROGRAMA NACIONA DE EDUCACIÓN ALTERNATIVA -PRO...,95.049505
739,PROGRAMA NACIONA DE EDUCACIÓN ALTERNATIVA -PRO...,"PROGRAMA NACIONAL DE EDUCACIÓN ALTERNATIVA, PR...",95.049505


In [35]:
def limpiar_establecimiento(df, columna="ESTABLECIMIENTO"):
    """
    Limpia y normaliza los valores de una columna de nombres de establecimientos educativos.

    Parámetros:
    ----------
    df : pandas.DataFrame
        DataFrame que contiene la columna a limpiar.
    columna : str, opcional (por defecto "ESTABLECIMIENTO")
        Nombre de la columna que se quiere limpiar.

    Proceso de limpieza:
    --------------------
    1. Convierte todos los valores a mayúsculas y normaliza tildes/acentos.
    2. Reemplaza guiones y guiones bajos por espacios.
    3. Reduce múltiples espacios a uno solo y elimina espacios al inicio y al final.
    4. Quita puntos finales.
    5. Elimina comillas dobles innecesarias y corrige espacios dentro de comillas.
    6. Quita espacios dentro de paréntesis.
    7. Aplica correcciones ortográficas y de tildes definidas en diccionarios.
    8. Quita comas en números (ej: "2,000" → "2000").
    9. Normaliza formato de "NO." seguido de número (ej: "NO1" → "NO. 1").
    10. Corrige espacios alrededor de comas (quita espacios antes, deja uno después).
    11. Limpieza final de espacios extra.

    Retorna:
    --------
    pandas.DataFrame
        DataFrame con la columna especificada limpia y normalizada.
    """

    # Función auxiliar para normalizar acentos y tildes
    def normalizar_tildes(texto):
        return unicodedata.normalize("NFC", texto)

    # Diccionario de correcciones ortográficas seguras (errores claros)
    correcciones_seguras = {
        "INSDUSTRIAL": "INDUSTRIAL",
        "BILINGUE": "BILINGÜE"
    }

    # Diccionario de correcciones de tildes
    correcciones_tildes = {
        "BILINGÚE": "BILINGÜE",
        "BINLINGÜE": "BILINGÜE",
        "TECNOLGÍA": "TECNOLOGÍA",
        "TECNOLOGIA": "TECNOLOGÍA",
        "TECNÓLOGICO": "TECNOLÓGICO",
        "EDCACIÓN": "EDUCACIÓN",
        "JÉSUS": "JESÚS",
        "EVANGELICO": "EVANGÉLICO",
        "COMPUTACION": "COMPUTACIÓN",
        "INFORMATICA": "INFORMÁTICA",
        "TECNICAS": "TÉCNICAS",
        "FORMACION": "FORMACIÓN",
        "BASICA": "BÁSICA"
    }

    # --- PROCESO DE LIMPIEZA ---
    df[columna] = (
        df[columna]
        .astype(str)                                # Asegurar tipo string
        .str.upper()                                # Convertir a mayúsculas
        .apply(normalizar_tildes)                   # Normalizar tildes
        .str.replace(r"[-_]", " ", regex=True)      # Guiones y guiones bajos -> espacio
        .str.replace(r"\s+", " ", regex=True)       # Un solo espacio
        .str.strip()                                # Quitar espacios extremos
        .str.rstrip(".")                            # Quitar punto final
        .str.replace(r'^"+', '', regex=True)        # Quitar comillas dobles al inicio
        .str.replace(r'"+$', '', regex=True)        # Quitar comillas dobles al final
        .str.replace(r'""', '"', regex=True)        # Comillas dobles internas -> una
        .str.strip()
        .str.replace(r'" +', '"', regex=True)       # Quitar espacio después de abrir comillas
        .str.replace(r' +"', '"', regex=True)       # Quitar espacio antes de cerrar comillas
        .str.replace(r"\(\s*(.*?)\s*\)", r"(\1)", regex=True)  # Quitar espacios en paréntesis
        .replace(correcciones_seguras, regex=True)  # Aplicar correcciones seguras
        .replace(correcciones_tildes, regex=True)   # Aplicar correcciones de tildes
        .str.replace(r"(\d),(\d{3})", r"\1\2", regex=True)      # Quitar comas en números
        .str.replace(r"\bNO\.?\s*(\d+)", r"NO. \1", regex=True) # Normalizar NO.
        .str.replace(r"(NO\. \d+),", r"\1, ", regex=True)       # Espacio después de coma en NO.
        .str.replace(r"\s+,", ",", regex=True)      # Quitar espacio antes de coma
        .str.replace(r",\s*", ", ", regex=True)     # Un espacio después de coma
        .str.replace(r"\s+", " ", regex=True)       # Quitar espacios extra
        .str.strip()
    )

    return df


# Aplicar la función de limpieza a la columna "ESTABLECIMIENTO"
df_final = limpiar_establecimiento(df_final, "ESTABLECIMIENTO")
